In [5]:
import shapely
import geopandas as gpd
import fiona
from shapely.geometry import Polygon

In [6]:
df_ride = gpd.read_file('./Input/Ride_1.shp')
df_ows = gpd.read_file('./Input/TLV_OWS_s.shp') #each row represents a street

In [7]:
df_ride = df_ride.to_crs({'init': 'epsg:3395'})
df_ows = df_ows.to_crs({'init': 'epsg:3395'})
df_ride_buffer = df_ride.copy()
df_ows_buffer = df_ows.copy()
df_ride_buffer.geometry = df_ride.buffer(-10, single_sided=True) #add buffer to the street layer and the ride lay on different sides
df_ows_buffer.geometry = df_ows.buffer(10, single_sided=True)    # note that each buffer is actually a polygon

/Users/mariapaulafernandez/Desktop/vcomm-dashboard/venv/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/mariapaulafernandez/Desktop/vcomm-dashboard/venv/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [20]:
df_ows_buffer_sorted = gpd.GeoDataFrame(columns = ['shem_angli', 'UniqueId', 'Shape_len','geometry']) #create an empty dataframe
# map the ride buffer to its closest street buffer and store it into the empty dataframe. Store the street id to the corresponding ride buffer
for i in range(0, len(df_ride_buffer)):
    min = df_ride_buffer.geometry[i].distance(df_ows_buffer.geometry[0])
    min_index = 0
    for j in range(0, len(df_ows_buffer)):
        if df_ride_buffer.geometry[i].distance(df_ows_buffer.geometry[j]) < min:
            min = df_ride_buffer.geometry[i].distance(df_ows_buffer.geometry[j])
            min_index = j
    df_ows_buffer_sorted.loc[i,:] = df_ows_buffer.loc[min_index,:]
df_ows_buffer_sorted.head()

In [10]:
df_ows_buffer_sorted = df_ows_buffer_sorted.set_crs(3395, allow_override=True)
df_intersection = gpd.GeoDataFrame(columns = ['shem_angli', 'UniqueId', 'Shape_len','geometry']) #create an empty dataframe and store the intersected buffer in it(if there is no intersection it will still store an empty polygon)
df_intersection = df_intersection.set_crs(3395, allow_override=True)
df_intersection.loc[:,['shem_angli', 'UniqueId', 'Shape_len']] = df_ows_buffer_sorted.loc[:,['shem_angli', 'UniqueId', 'Shape_len']]
df_intersection.geometry = df_ride_buffer.intersection(df_ows_buffer_sorted)

/Users/mariapaulafernandez/Desktop/vcomm-dashboard/venv/lib/python3.7/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


In [11]:
df_not_empty = gpd.GeoDataFrame(columns = ['UniqueId','geometry', 'area']) #create an empty dataframe and store the non empty polygon in it
df_not_empty = df_not_empty.set_crs(3395, allow_override=True)
counter = 0
for i in range(len(df_intersection)):
    if df_intersection.geometry[i].is_empty == False:
        df_not_empty.loc[counter,['UniqueId','geometry']] = df_intersection.loc[i,['UniqueId','geometry']]
        counter += 1

In [16]:
# calculate each polygon's area and append it to the area column
for i in range(len(df_not_empty)):
    df_not_empty['area'][i] = df_not_empty.geometry.loc[i].area
    for j in range(i+1, len(df_not_empty)):
        if df_not_empty['UniqueId'][i] == df_not_empty['UniqueId'][j]:
            df_not_empty['area'][i] += df_not_empty.geometry.loc[j].area
            df_not_empty.drop(index=[j], inplace=True)
    df_not_empty.reset_index(drop=True, inplace=True) 

In [17]:
# group the ride buffer by its street id and sum up the area. Compare the area to its corresponding street buffer area. If the the ratio > 0.5 then it is a violation
counter = 0
ID = gpd.GeoDataFrame(columns=['UniqueId'])
for i in range(len(df_not_empty)):
    for j in range(len(df_ows_buffer)):
        if df_not_empty.UniqueId[i] == df_ows_buffer.UniqueId[j]:
            area = df_ows_buffer.geometry[j].area
            if df_not_empty['area'][i]/area > 0.5:
                ID.loc[counter,:] = df_not_empty.loc[i,['UniqueId']]
                counter += 1

In [19]:
# output the streets that the rider has violated the safety event
list_id=list(ID['UniqueId'])
df_result = df_ows.loc[df_ows['UniqueId'].isin(list_id)]
df_result.reset_index(drop=True,inplace=True)
df_result.set_crs(3395, allow_override=True)

,shem_angli,UniqueId,Shape_len,geometry
